In [25]:
import numpy as np
import pandas as pd

In [26]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv') 

In [27]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [28]:
movies.shape

(4803, 20)

In [29]:
movies = movies.merge(credits,on='title')

In [30]:
movies.rename(columns={'id': 'movie_id', 'cast_x': 'cast','crew_x': 'crew'}, inplace=True)


In [31]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [32]:
movies = movies.loc[:, ~movies.columns.duplicated()]


In [9]:
movies = movies.copy()
movies.dropna(inplace=True)

In [10]:
movies.duplicated().sum()

0

In [11]:
import ast

In [12]:
def convert (obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [13]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L 

In [14]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L 

In [15]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [16]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [17]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" "," ") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" "," ") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" "," ") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" "," ") for i in x])

In [18]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [19]:
new_df = movies[['movie_id','title','tags']]

In [20]:
import nltk

In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [22]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [24]:
vectors = cv.fit_transform(new_df['tags']).toarray()

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
vectors.shape

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(vectors).shape

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances= similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x: x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
        

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movie_list.pkl','wb'))

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("sklearn:", sklearn.__version__)
print("ast: Built-in module (No version)")